# Introdução à Inteligência Artificial (2022/2023)

## Planeamento

### Conteúdos

* Afirmações lógicas
* Classe PlanningProblem
    * o conhecimento base
    * o objetivo
    * a classe Action
* Exemplos de formulação de problemas:
    * Torre com 3 blocos
    * Viagem à Roménia
    * Comer bolo
* Procura em espaço de estados
    * Procura progressiva (Forward)
* Exercícios


## Introdução

Nesta aula vamos formular problemas de planeamento usando a linguagem Python. Como o planeamento combina duas áreas de IA: procura e lógica, é necessário primeiro perceber como se escrevem afirmações lógicas em Python, antes de começarmos a formular problemas de planeamento.


### Recursos necessários
Para este guião os seguintes módulos são necessários (distribuídos juntamento com o guião):
* `planningPlus.py` - módulo principal
* `search.py` - módulo principal
* `logic.py` - módulo auxiliar
* `utils.py` - módulo auxiliar

### Convenções sintáticas

A utilização deste planeador de Python necessita de algumas convenções, para se definirem predicados, termos, e variáveis. Assim,

* Inicial maiúscula para termos e predicados (p.ex., Table, Block(A), On(A,Table), ...)
* Inicial minúscula para variáveis (p.ex., x, y, a, ...)

## Módulo *planningPlus.py* - breve explicação

Este módulo é uma variante muito ligeira do *planning.py* que está disponível no repositório aima-python, que contém a implementação (em Python) da generalidade dos algoritmos descritos no livro da disciplina (Russel & Norvig). Muitas das definições deste módulo não serão utilizadas. Vamos apenas concentrar-nos em algumas das suas classes e funções. No essencial, é disponibilizado o seguinte:

* A classe **PlanningProblem** que vamos utilizar para definir problemas de planeamento.
* A classe **Action** que é usada para representar um esquema de ação.
* As classes **ForwardPlan** e **BackwardPlan** para definir problemas de espaços de estados a serem usados nos algoritmos de procura.

Para a parte dedicada à formulação, apenas precisamos de usar as classes **PlanningProblem** e **Action** do módulo, que tem de ser importado, mas não deverão alterá-lo.

In [ ]:
from planningPlus import *
from utils import *
from search import *

## O problema da torre com 3 blocos

O dominio deste problema consiste num conjunto de blocos em cima de uma mesa. Os blocos podem ser empilhados, mas apenas um bloco pode ser colocado diretamente em cima de outro. Um robot pode pegar num bloco e movê-lo para outra posição, em cima da mesa ou em cima de outro bloco. O robot apenas pode pegar num bloco de cada vez, o que significa que não pode pegar num bloco que tenha outro em cima. O objetivo é construir uma torre de blocos. A configuração que iremos usar neste exemplo também é conhecida como Anomalia de Sussman.

<img src="3-blocks.png" width="300">

Pretendemos formular este problema como um problema de planeamento usando PDDL (*Planning Domain Definition Language*). Para definirmos um problema temos de ter:

* um conhecimento base
* um conjunto de objetivos
* um conjunto de esquemas de ações

Cada conhecimento base é representado por um conjunto de expressões que descrevam o estado inicial do problema. Essa descrição é feita recorrendo a proposições lógicas de primeira ordem. A representação dos estados é cuidadosamente projetada para que um estado possa ser tratado como uma conjunção de expressões, que pode ser manipulada por inferência lógica, ou como um conjunto de expressões, que podem ser manipuladas com operações definidas.

Antes de descrevermos o nosso problema dos blocos, precisamos de perceber como trabalhar com expressões lógicas em Python. 

## Afirmações lógicas

A classe `Expr` é usada para representar qualquer tipo de expressão lógica. O tipo mais simples de `Expr` que se pode definir usa a função `Symbol`:

In [ ]:
Symbol('X')

Ou pode-se definir múltiplos simbolos ao mesmo tempo com a função `symbols`:

In [ ]:
(x, y, P, Q, f) = symbols('x, y, P, Q, f')

Pode-se combinar objetos `Expr` com os operadores de Python (&, ~, |, ...). Se quisessemos formar a afirmação "P e não Q", seria da seguinte forma. Repare que P e Q são simbolos definidos anteriormente.

In [ ]:
P & ~Q

Isto funciona porque a classe `Expr` sobrepõe o operador `&` (*overloading*) com a definição:
```python
def __and__(self, other): return Expr('&',  self, other)
```
e faz sobreposições semelhantes para outros operadores. Um objeto `Expr` tem dois atributos:

* `op` para o operador, que é sempre uma string
* `args` para os argumentos, que é um tuplo com zero ou mais expressões

Por "expressões" entende-se uma instância de `Expr`, ou um número. 

Se não definirmos os simbolos não é possivel usar os operadores de Python para escrever expressões diretamente.

In [ ]:
A & B

Vamos verificar os atributos de alguns exemplos de objetos `Expr`:

In [ ]:
sentence = P & ~Q
sentence.op

In [ ]:
sentence.args

In [ ]:
P.op

In [ ]:
P.args

Também se podem definir predicados, como por exemplo

In [ ]:
Pxy = P(x, y)
Pxy.op

In [ ]:
Pxy.args

É importante salientar que a classe `Expr` apenas fornece uma forma de "representar" expressões. Cada um dos `args` num objeto `Expr` pode ser um símbolo, um número, ou outra `Expr`. Vejemos um exemplo onde usamos números.

De seguida apresenta-se uma tabela de operadores que podem ser usados para criar afirmações. Mas existe um problema: queremos usar os operadores de Python para construir afirmações, mas o Python não permite a seta de implicação como operador. Assim, teremos de usar uma notação ligeiramente diferente que o Python permita: `|'==>'|` em vez de apenas `==>`. Em alternativa, pode-se usar a forma mais verbosa do construtor de `Expr`:

| Operação                 | Livro | Input Python | Output Python | Input `Expr`
|--------------------------|----------------------|-------------------------|---|---|
| Negação                 | &not; P      | `~P`                       | `~P` | `Expr('~', P)`
| E                      | P &and; Q       | `P & Q`                     | `P & Q` | `Expr('&', P, Q)`
| Ou                       | P &or; Q | `P`<tt> &#124; </tt>`Q`| `P`<tt> &#124; </tt>`Q` | `Expr('`&#124;`', P, Q)`
| Desigualdade         | P &ne; Q     | `P ^ Q`                | `P ^ Q`  | `Expr('^', P, Q)`
| Implicação                  | P &rarr; Q    | `P` <tt>&#124;</tt>`'==>'`<tt>&#124;</tt> `Q`   | `P ==> Q` | `Expr('==>', P, Q)`
| Implicação contrária      | Q &larr; P     | `Q` <tt>&#124;</tt>`'<=='`<tt>&#124;</tt> `P`  |`Q <== P` | `Expr('<==', Q, P)`
| Equivalência            | P &harr; Q   | `P` <tt>&#124;</tt>`'<=>'`<tt>&#124;</tt> `Q`   |`P <=> Q` | `Expr('<=>', P, Q)`

In [ ]:
~(P & Q)  |'==>'|  (~P | ~Q)

Equivalentemente,

In [ ]:
Expr('==>', ~(P & Q), ~P | ~Q)

Existe uma forma mais simples de definir afirmações, usando a função `expr`. Esta função tem como input uma string, e faz "parsing" para um objeto `Expr`. A string pode conter os operadores setas: `==>`, `<==`, ou `<==>`, que serão tratados como se fossem operadores de Python. Além disso, `expr` define automaticamente quaisquer dos respectivos simbolos, não é necessário serem pré-definidos (como fizemos com o P e Q).

In [ ]:
expr('~(P & Q) ==> (~P | ~Q)')

Se quisermos escrever a expressão "Pencil at Table", usamos predicados da seguinte forma

In [ ]:
f = expr('At(Pencil,Table)')

O operador e os argumentos são respetivamente

In [ ]:
f.op

In [ ]:
f.args

Mais alguns exemplos de predicados que se podem escrever:

In [ ]:
# disc on pin
expr('On(Disc, Pin)')

In [ ]:
# supermarket (SM) sells oranges
expr('Sells(SM, Oranges)')

In [ ]:
# 1 is preceded by 2
expr('precededBy(1,2)')

In [ ]:
# 2 is not preceded by 2
expr('~precededBy(2,1)')

## A classe *PlanningProblem*

A classe **PlanningProblem** é usada para representar problemas de planeamento. Os seguintes atributos são essenciais para se conseguir definir um problema:

* um conhecimento base (*initial*)
* objetivos (*goals*)
* conjunto de esquemas de ações viáveis (*actions*)
* o domínio do problema (*domain*)

A seguir apresenta-se o construtor desta classe:

```python
def __init__(self, initial, goals, actions, domain=None):
        self.initial = self.convert(initial) if domain is None else self.convert(initial) + self.convert(domain)
        self.goals = self.convert(goals)
        self.actions = actions
        self.domain = domain
```

##### O domínio

O atributo *domain* do problema é um conjunto de expressões que descrevem a informação estática, i.e., contém informação sobre os tipos, propriedades e relações entre os objetos representados, informação essa que não é alterada com as acções, e também não será alterada durante o processo de procura da solução.

Vamos tentar descrever o domínio para o problema dos blocos. Para tal é necessário criar predicados que descrevam os objetos.
<img src="3-blocks.png" width="300">

Sabemos que temos três blocos (designemo-los por A, B e C) e uma mesa. Então podemos definir o predicado formado por um argumento e o operador 'Block', que indica que um determinado objecto é do tiplo bloco. Por exemplo, o domínio irá conter a expressão seguinte, para o bloco A:

In [ ]:
expr('Block(A)')

Apenas estamos a considerar os três blocos para o domínio e não consideramos a mesa. Aqui poderíamos ter três alternativas: dizer que Table é mesa (p.ex., `Table(Table)`), dizer que Table não é bloco (`~Block(Table)`), ou então não dizer nada. Apresentamos o terceiro caso, pois esse objeto não irá sofrer alteração nenhuma durante a aplicação das ações, i.e., nunca usaremos `Table(x)` ou `~Block(x)` nas pré-condições, ou no domínio ou nos efeitos dos esquemas de ações. Ao contrário dos blocos que irão ser colocados em cima da mesa ou em cima de outro bloco. De facto, precisaremos apenas de restringir as variáveis das ações apenas para os blocos, e assim exigiremos que algumas variáveis do tipo `x` satisfaçam `Block(x)` e não serão instanciados para `x=Table`. Se tivessemos mais do que uma mesa e pudessemos transferir blocos entre mesas, então daria jeito termos `Table(x)`.

Assim, o domínio deste problema é dado por:

In [ ]:
expr('Block(A) & Block(B) & Block(C)')

##### O conhecimento base

O atributo *initial* é uma conjunção de expressões que descrevem o conhecimento base do problema. Em qualquer situação, se uma expressão não foi previamente afirmada, então não é conhecida.

Vamos tentar descrever o conhecimento base para o problema dos blocos. Para tal é necessário criar operadores que descrevam o ambiente. Por exemplo, sabemos que o bloco A está em cima da mesa ("A on Table"), e então podemos descrever esta situação usando a seguinte expressão:

In [ ]:
expr('On(A, Table)')

Para que o robô possa pegar num bloco é necessário sabermos que o bloco que se pega não tem nenhum outro por cima. O mesmo acontece quando se quer colocar um bloco em cima de outro, este último tem de estar livre. Vamos arranjar um predicado com o operador `Clear` e um argumento para representar essa informação. Na situação inicial, sabemos que por cima do bloco C não existe nada, que o podemos representar pela expressão:

In [ ]:
expr('Clear(C)')

A mesa (Table) é infinita, e haverá sempre espaço para lá colocar mais um bloco, mesmo que tivessemos uma centena de blocos. Assim, é desnecessário estar a afirmar `Clear(Table)`. Poderíamos fazê-lo mas por uma questão de parsimónia (reduzir ao máximo o que é redundante) não o fazemos.

Assim o conhecimento base do problema (ver figura) pode ser definido diretamente, sem necessitar de previamente terem sido definidos estes predicados, como sendo:

In [ ]:
expr('On(A, Table) & On(B, Table) & On(C, A) & Clear(B) & Clear(C)')

##### Exercício 1
Como exprimiria o conhecimento de uma situação em que temos os mesmos 3 blocos, mas cada um deles sobre a mesa?

In [ ]:
### Solução do exercício 1



##### O objetivo

O atributo *goals* é uma expressão ou conjunção de expressões (positivas ou negativas) que indicam os objetivos a atingir no problema. No caso deste problema dos blocos, queremos ter o A por cima do B, o B por cima do C e C em cima da mesa. Assim, queremos a expressão

In [ ]:
expr('On(B, C) & On(A, B)')

Notem também que não temos `On(C,Table)`. Poderíamos fazê-lo mas não parece ser fundamental. Existindo 3 blocos apenas, em que B fica em cima de C e A em cima de B, necessariamente C terá de ficar na mesa, sem termos de o exprimir. Por isso foi omitido do objectivo.

Notem que poderíamos também incluir `Clear(A)`na conjunção, mas não é de novo fundamental. A questão é que pelo facto de existirem apenas 3 blocos, A necessariamente ficará livre quando estiver sobre B, e quando este está sobre C, e esse predicado será atingido quando os dois da conjunção em cima forem obtidos.

### Exercício 2
Exprimam um novo objetivo do problema dos blocos, considerando que apenas queremos que o bloco B passe a ter um outro bloco por cima.

In [ ]:
# Solução do exercício 2



### Exercício 3
Exprimam um novo objetivo do problema dos blocos, considerando que apenas queremos que o bloco B esteja em cima de qualquer outro bloco.

In [ ]:
# Solução do exercício 3



##### Esquemas de ações

O atributo *actions* contém uma lista de objetos da classe `Action` que representam esquemas de ações que posteriormente podem ser executadas no espaço de procura do problema. Uma instância da classe `Action` é definida pelo nome da ação, uma lista de variáveis usadas na ação, pré-condições, efeitos e domínio. Vamos olhar para o construtor desta classe:

```python
def __init__(self, action, precond, effect, domain=None):
        if isinstance(action, str):
            action = expr(action)
        self.name = action.op
        self.args = action.args
        self.precond = self.convert(precond) if domain is None else self.convert(precond) + self.convert(domain)
        self.effect = self.convert(effect)
        self.domain = domain
```

Esta classe representa um esquema de ação que inclui uma expressão, um domínio, as pré-condições, e os efeitos após a ação ser aplicada. 
* Os atributos *name* e *args* provêm do argumento *action* do construtor, o qual é uma instância de `Expr`, ou seja, tem um operador e argumentos (como visto anteriormente). 
* O atributo *precond* é uma conjunção de expressões (positivas ou negativas) com as pré-condições que devem ser verificadas no estado para se poder aplicar a ação.
* o atributo *effect* é uma conjunção de expressões com os efeitos/alterações ao estado após a aplicação da ação.
* O atributo *domain* descreve a informação estática do problema associada a uma ação. Evita aumentar as pré-condições com conhecimento estático. A definição deste atributo irá ser importante para a eficiência dos planeadores quando têm ações com variáveis para filtrar as permutações dos objetos do problema que podem gerar regras concretas a partir das regras genéricas.

As condições e efeitos negativos são introduzidos usando o simbolo `~` antes da expressão, que internamente é codificado com um `Not` e colocado como prefixo da expressão. Por exemplo, a negação de `At(obj, loc)` será introduzido como `~At(obj, loc)` e internamente representado por `NotAt(obj, loc)`. O método `convert` da classe `Action`, para além de converter as negações, pega numa string e faz "parsing", removendo as conjunções (se existirem) e retorna uma lista de objetos `Expr`. 

Ainda nesta classe, existem outros métodos, incluindo o `check_precond`. Este método verifica se as pré-condições para uma dada ação são válidas, dado um estado. 

**IMPORTANTE** É preciso salientar que variáveis diferentes correspondem sempre a objetos diferentes, não sendo necessário incluir expressões do género `b != x` nas pré-condições.

### Esquemas de acções no Problema dos Blocos
Vamos então tentar descrever os esquemas de ações do nosso problema dos blocos. Quando descrevemos um esquema de ação, descrevemos de forma genérica, usando variáveis, para que a ação possa ser aplicada a qualquer um dos objetos do problema. Neste problema existem dois tipos de ações: `MoveToTable` e `Move`.

* **`MoveToTable(b, x)`** -- mover o bloco `b` que está em cima de `x` para a mesa.
    * Este esquema de acção apenas se aplica a dois blocos porque não pretendemos mover um objeto que está em cima da mesa para a mesa, nem queremos mover uma mesa que esteja em cima de alguma coisa para a mesa, o que significa que nem `b` nem `x` podem ser mesa, terão de ser blocos. 
    * nas pré-condições é obrigatório que não exista nada em cima do bloco `b` e é preciso que `b` esteja sobre `x`.
    * nas pós-condições sabemos que o bloco `x` passará a estar livre e que `b` passará a estar sobre a mesa e deixará de estar sobre `x`.
* **`Move(b, x, y)`** -- mover o bloco `b` que está em `x` para cima de `y`. Neste esquema queremos mover um bloco que pode estar em cima de outro ou em cima da mesa, para cima de outro bloco. 
    * Assim, no domínio não se coloca `Block(x)` porque `x` pode ser a mesa, mas exige-se `Block(b)` e `Block(y)`.
    * Nas pré-condições é necessário desde que ambos `b` e `y` não tenham nada por cima.
    * Nas pós-condições teremos que `x` passará a estar livre, que `y` deixará de estar livre e que `b` passará a estar em cima de `y` e não em cima de `x`.
    
Os domínios das acções levam a que as ações instanciadas com `MoveToTable(Table, x)`, `MoveToTable(x, Table)`, `Move(Table, x, y)` e `Move(x, y, Table)`, em que `x` e `y` são blocos A, B, C ou Table, nunca serão geradas no processo de planeamento.

Assim o esquema de ações será uma lista de objetos `Action`, um objeto para cada esquema de ação que se queira definir.

In [ ]:
acoes = [Action('MoveToTable(b, x)',
                precond='On(b, x) & Clear(b)',
                effect='On(b, Table) & Clear(x) & ~On(b, x)',
                domain='Block(b) & Block(x)'),
         Action('Move(b, x, y)',
                precond='On(b, x) & Clear(b) & Clear(y)',
                effect='On(b, y) & Clear(x) & ~On(b, x) & ~Clear(y)',
                domain='Block(b) & Block(y)')]
print(acoes)

### Formular o problema dos blocos

Vamos então formular o problema dos blocos, usando tudo o que aprendemos. Para o conhecimento base e o objetivo, não iremos usar o `expr` porque a classe *PlanningProblem* (tal como a classe *Action*) também possui um método `convert` para converter strings em objetos `Expr`.

In [ ]:
# conhecimento base
kb = 'On(A, Table) & On(B, Table) & On(C, A) & Clear(B) & Clear(C)'
# dominio
dominio = 'Block(A) & Block(B) & Block(C)'
# objetivo
goal = 'On(B, C) & On(A, B)'

threeBlockTower = PlanningProblem(initial=kb, goals=goal, actions=acoes, domain=dominio)

Olhemos para o estado inicial

In [ ]:
print(threeBlockTower.initial)

Antes de aplicar alguma ação a este problema, vamos verificar se este problema atingiu o objetivo:

In [ ]:
threeBlockTower.goal_test()

Como se pode verificar, ainda não atingiu o objetivo. Vamos experimentar mais alguns métodos das classes. Por exemplo, ver quais os esquemas de ações que se podem aplicar:

In [ ]:
for a in threeBlockTower.actions:
    print(a)

### Instanciação dos esquemas de acções no Mundo dos Blocos

Vamos então instanciar os esquemas de acções, usando o método `expand_actions` da classe `Planning_Problem`. Reparem que o campo *domain* na definição dos esquemas tem um papel fundamental na concretização das acções.
Estas são todas as acções possíveis dados os 3 blocos

In [ ]:
threeBlockTower.expand_actions()

Importa sublinhar a importância do domínio do planeador e a sua interacção com o domínio do problema. Reparem que só as expressões concretizadas do domínio do problema que envolvem os termos dos próprios blocos é que satisfazem os domínios dos 2 esquemas de acções dados em termos de variáveis..

Na prática, existem quatro objetos definidos: A, B, C e Table, e todas as permutações destes objetos irão ser testadas em cada um dos esquemas de ações. No entanto, apenas algumas ações serão válidas por causa da filtragem do dominio, que só aceita alguns objetos como sendo blocos.

Notem que as ligações entre os objectos do mundo e as variáveis (*bindings*), na permutação, e os parâmetros da acção, são primeiro filtradas pelos domínios e se os satisfizerem então são propagadas tanto para as pré-condições como para os efeitos.

Vamos expandir as acções de novo com o modo verboso ligado:

In [ ]:
threeBlockTower.expand_actions(verbose=True)

Se omitirmos o bloco C no domínio do problema, como apresentamos a seguir:

In [ ]:
# conhecimento base
kb = 'On(A, Table) & On(B, Table) & On(C, A) & Clear(B) & Clear(C)'
# dominio
dominio = 'Block(A) & Block(B)'
# objetivo
goal = 'On(B, C) & On(A, B)'

threeBlockTower = PlanningProblem(initial=kb, goals=goal, actions=acoes, domain=dominio)

ao expandirmos as acções, fazendo

In [ ]:
threeBlockTower.expand_actions()

obtivemos muitos menos acções. Afinal, desapareceram todas que impliquem mover o bloco C ou mover para cima do bloco C, porque C deixou de ser considerado como bloco ao desaparecer `Block(C)` do domínio do problema.

E se tivermos um domínio mal definido nas acções, podemos ter um conjunto de acções que não fazem sentido. 

Por exemplo, retiremos `Block(b)` do domínio da acção `MoveToTable` e obteremos acções em que se move a `Table` de um bloco para a `Table` entre outras acções incorrectas. Vejam que embora seja gerada a partir do esquema, essa acção nunca poderia ser realizada porque nunca seria satisfeita uma das suas pré-condições: `Clear(Table)`.

In [ ]:
acoes = [Action('MoveToTable(b, x)',
                precond='On(b, x) & Clear(b)',
                effect='On(b, Table) & Clear(x) & ~On(b, x)',
                domain='Block(x)'),
         Action('Move(b, x, y)',
                precond='On(b, x) & Clear(b) & Clear(y)',
                effect='On(b, y) & Clear(x) & ~On(b, x) & ~Clear(y)',
                domain='Block(b) & Block(y)')]
print(acoes)

threeBlockTower = PlanningProblem(initial=kb, goals=goal, actions=acoes, domain=dominio)

threeBlockTower.expand_actions()

Verifiquemos o que acontece se esses dominios não forem definidos, i.e., a definição de bloco for integrada nas pré-condições dos esquemas de ações. Vamos definir agora o problema *blockTower* onde o esquema de ações não tem o atributo domínio explicitamente definido.

In [ ]:
# ações
ac = [Action('MoveToTable(b, x)',
                precond='On(b, x) & Clear(b) & Block(b) & Block(x)',
                effect='On(b, Table) & Clear(x) & ~On(b, x)'
               ),
         Action('Move(b, x, y)',
                precond='On(b, x) & Clear(b) & Clear(y) & Block(b) & Block(y)',
                effect='On(b, y) & Clear(x) & ~On(b, x) & ~Clear(y)'
               )]

blockTower = PlanningProblem(initial=kb, goals=goal, actions=ac, domain=dominio)

In [ ]:
blockTower.expand_actions()

Reparem que foram instanciadas 36 ações (o dobro das ações instanciadas no problema *threeBlockTower*), ações como por exemplo `MoveToTable(B, Table)` (i.e., mover o bloco B que está em cima da mesa para a mesa) a qual não faz nenhum sentido, sendo uma acção totalmente redundante.

Muitas destas ações não serão utilizadas porque as pré-condições irão falhar. No entanto, o planeador irá testá-las para saber quais poderá ou não aplicar a um estado, o que fará com que a procura seja muito mais demorada.

### Aplicação das acções e verificação de satisfação dos objectivos

Redefinamos então o problema dos blocos correctamente.

In [ ]:
# conhecimento base
kb = 'On(A, Table) & On(B, Table) & On(C, A) & Clear(B) & Clear(C)'
# dominio
dominio = 'Block(A) & Block(B) & Block(C)'
# objetivo
goal = 'On(B, C) & On(A, B)'

threeBlockTower = PlanningProblem(initial=kb, goals=goal, actions=acoes, domain=dominio)

Vamos aplicar a ação `MoveToTable(C,A)` que corresponde ao esquema de ação `MoveToTable(x,y)`, quando `x=C` e `y=A`, e verificar se atingiu o objetivo. Reparem que ao aplicarmos uma ação, através do método `act` o estado mudou, sendo o estado resultante guardado no atributo *initial* do problema.

In [ ]:
# conhecimento base
print('Conhecimento base:', threeBlockTower.initial)
# vamos aplicar uma ação ao estado inicial do problema
threeBlockTower.act(expr('MoveToTable(C,A)'))
print('Estado seguinte:', threeBlockTower.initial)

Ao movermos o bloco C, que está em cima do A, para a mesa, desapareceu do estado a expressão `On(C,A)` e foi adicionado `NotOn(C,A)`. Foram ainda adicionadas mais algumas expressões para descreverem o estado atual: 
* `Clear(A)` - o bloco A deixou de ter o bloco C em cima, i.e., passou a estar livre
* `On(C, Table)` - o bloco C passou a estar em cima da mesa

Vamos aplicar agora a ação `Move(C, Table, A)` e ver o que acontece.

In [ ]:
print('Estado atual:', threeBlockTower.initial)
threeBlockTower.act(expr('Move(C,Table,A)'))
print('Estado seguinte:', threeBlockTower.initial)

Reparem agora que voltamos a colocar o bloco C em cima do A, desaparecendo a expressão `NotOn(C,A)` e `On(C, Table)` e sendo substituidas por `On(C,A)` e `NotOn(C,Table)`, respetivamente.

### Exercício 4
Modifiquem o objetivo do problema dos blocos, considerando que apenas queremos que o bloco B passe a ter um outro bloco por cima. Apliquem ao estado inicial do problema a acção de mover o C para cima do B e confirmem se atingiu o objectivo.

In [ ]:
# Solução do exercício 4



Regressemos ao problema inicial:

In [ ]:
# conhecimento base
kb = 'On(A, Table) & On(B, Table) & On(C, A) & Clear(B) & Clear(C)'
# dominio
dominio = 'Block(A) & Block(B) & Block(C)'
# objetivo
goal = 'On(B, C) & On(A, B)'

threeBlockTower = PlanningProblem(initial=kb, goals=goal, actions=acoes, domain=dominio)

e vamos agora aplicar uma série de ações e verificar se atingimos o objetivo.

In [ ]:
solution = [expr('MoveToTable(C,A)'),
            expr('Move(B, Table, C)'),
            expr('Move(A, Table, B)')
           ]

for action in solution:
    threeBlockTower.act(action)

print('Atingiu o goal?', threeBlockTower.goal_test())

Vejemos o estado final

In [ ]:
print('Estado atual:', threeBlockTower.initial)

### Problemas da Formulação:
Notem que o predicado `Clear(Table)` aparece no estado e mais do que uma vez.

A acção `Move(x,y,z)` deveria ser apenas entre blocos de modo a que fizesse sentido o bloco y ficar livre, i.e., y não deveria ser a mesa (Table).

Deveriamos ter um outro esquema de acção específico para mover um bloco que estivesse na mesa para cima de outro bloco.

### Exercício 5
Formulem o problema de modo a resolver o problema da formulação indicado em cima, resultando em que não apareçam os predicados `Clear(Table)`. Apliquem as acções que levem ao estado final e confirmem que essas expressões não aparecem.

In [ ]:
# Solução do 5



## Outro Exemplo: Viagem à Roménia

Vamos tentar planear uma viagem  numa versão simplificada da Roménia. 
<img src="romania.png" width="250">

O conhecimento base indica apenas a nossa localização no mapa (*Sibiu*). A informação sobre que cidades estão interligadas por estradas e quais têm aeroportos é informação estática do nosso problema, sendo definida no dominio. No entanto, não é necessário representar cidades, como por exemplo, `City(Sibiu)`, porque todas são cidades. Para além disso, não iremos representar ligações simétricas entre cidades.

In [ ]:
# conhecimento base
kb = 'At(Sibiu)'
# dominio
prob_domain = 'Airport(Sibiu) & Airport(Bucharest) & Airport(Craiova) & Connected(Bucharest,Pitesti)' + \
                ' & Connected(Pitesti,Rimnicu) & Connected(Rimnicu,Sibiu) & Connected(Sibiu,Fagaras)' + \
                '& Connected(Fagaras,Bucharest) & Connected(Pitesti,Craiova) & Connected(Craiova,Rimnicu)'

Vamos agora descrever esquemas de ações para o problema. Sabemos que podemos ir de carro entre quaisquer lugares ligados. Mas como é evidente, também podemos ir de avião entre Sibiu, Bucharest e Craiova.

Vamos definir as ações de voar desta forma:

In [ ]:
# fly from any city with airport towards any other city with airport
precond = 'At(x)'
effect = 'At(y) & ~At(x)'
domain = 'Airport(x) & Airport(y)'
fly = Action('Fly(x, y)', precond, effect, domain)

E qualquer ação de conduzir desta forma:

In [ ]:
# Drive from x to y
precond = 'At(x)'
effect = 'At(y) & ~At(x)'
domain = 'Connected(x,y)'
drive = Action('Drive(x, y)', precond, effect, domain)

# Drive from y to x -- usa a ligação simétrica Connected(y,x)
precond = 'At(x)'
effect = 'At(y) & ~At(x)'
domain = 'Connected(y,x)'
drive_sim = Action('DriveInv(x, y)', precond, effect, domain)

O nosso objetivo é definido como chegar a Bucharest.

In [ ]:
goals = 'At(Bucharest)'

Assim, podemos definir o seguinte problema de planeamento:

In [ ]:
prob = PlanningProblem(kb, goals, [fly, drive, drive_sim], prob_domain)

Tal como anteriormente vamos testar se atingiu o objetivo e aplicar algumas ações a este problema.

In [ ]:
print('Objetivo?', prob.goal_test())

In [ ]:
print('Conhecimento base:', prob.initial)

Reparem que o dominio do problema é sempre adicionado ao conhecimento base.

Vamos agora aplicar uma ação de voar entre Sibiu e Bucharest.

In [ ]:
prob.act(expr('Fly(Sibiu, Bucharest)'))
print('Estado após aplicar ação de voar:', prob.initial)

Vamos agora assumir que estamos em Pitesti e queremos viajar para Bucharest. Reparem que não existem voos entre estas duas cidades, o que significa que terá de ser uma viagem por carro.

In [ ]:
prob = PlanningProblem('At(Pitesti)', goals, [fly, drive, drive_sim], prob_domain)

Vamos aplicar a ação `Drive` entre Pitesti e Bucharest. Reparem que no nosso *knowledge_base* definimos a ligação `Connected(Bucharest, Pitesti)`, mas não ao contrário. O que significa que iremos usar a segunda ação `DriveInv(Pitesti, Bucharest)`.

In [ ]:
prob.act(expr('DriveInv(Pitesti,Bucharest)'))
print('Estado após aplicar ação de conduzir:', prob.initial)

In [ ]:
print('Objetivo?',prob.goal_test())

## Mais um exemplo: Comer bolo

Vamos formular um problema de comer um bolo, i.e., o agente quer comer um bolo (este é objetivo). O que significa que se o agente tem um bolo, come-o. Se não tem um bolo, faz um, e ao fazer um bolo, passa a ter um e, consequentemente, pode comê-lo. Vamos assumir que inicialmente não existe bolo. 

Este é um exemplo muito simples onde se pretende mostrar que é possivel formular problemas de planeamento cujo o conhecimento base e as pré-condições têm expressões negadas. 

In [ ]:
inicial = '~Have(Cake)'
goal = 'Eaten(Cake)'

As ações serão comer um bolo `Eat(Cake)` e fazer um bolo `Bake(Cake)`. Aqui temos que ter atenção às pré-condições que iremos definir. Só poderemos fazer um bolo se não existir um bolo.

In [ ]:
actions = [Action('Eat(Cake)',
                  precond='Have(Cake)',
                  effect='Eaten(Cake) & ~Have(Cake)'),
           Action('Bake(Cake)',
                  precond='~Have(Cake)',
                  effect='Have(Cake)')]

In [ ]:
cake = PlanningProblem(initial=inicial, goals=goal, actions=actions)

In [ ]:
cake.initial

A ordem pela qual aplicamos as ações é importante, pois não podemos aplicar a ação de comer um bolo se esse bolo não existir, ou seja, tem de se fazer o bolo para que a pré-condição da ação `Eat(Cake)` seja satisfeita.

In [ ]:
cake.act(expr('Eat(Cake)'))

Vamos aplicar a ação de fazer um bolo e depois comer o bolo, para verificarmos o objetivo.

In [ ]:
print('Estado inicial:', cake.initial)
cake.act(expr('Bake(Cake)'))
print('Estado após ação fazer bolo:', cake.initial)

Reparem que o estado atual consiste em ter feito um bolo, logo pode ser aplicada a ação de comer o bolo, a pré-condição está satisfeita.

In [ ]:
cake.act(expr('Eat(Cake)'))
print('Estado após ação comer bolo:', cake.initial)
print('Atingiu objetivo?', cake.goal_test())

## Exercício 6

Consideremos o problema de trocar um pneu furado num carro. O pneu furado (`Flat`) encontra-se no eixo do carro (`Axle`) e o pneu sobressalente (`Spare`) na bagageira (`Trunk`). O objetivo é colocar o pneu sobressalente no eixo do carro e o pneu furado no chão (`Ground`). Formule este problema como um problema de planeamento.

**Dica:** Temos como dominio do problema os dois pneus (`Tire(Flat) & Tire(Spare)`). Iremos ter duas ações: `Remove` para remover um objeto de uma localização (p.ex. remover o pneu do eixo ou da bagageira) e `PutOn` para colocar um pneu no eixo.

In [ ]:
# Resolução do exercicio 6



## Planeamento como procura em espaços de estados

A descrição de um problema de planeamento define um problema de procura: podemos procurar a partir do estado inicial através de espaços de estados, procurando o objetivo. Uma das vantagens da representação declarativa dos esquemas de ações é que também se pode procurar "backward" a partir do objetivo, procurando o estado inicial.

As classes `ForwardPlan` e `BackwardPlan` são duas subclasses da classe `Problem`, que foi usada na formulação de problemas de espaços de estados. Assim, ambas as subclasses têm definidos métodos já conhecidos, nomeadamente:
* `actions(self, state)` - este método, dado um estado, devolve a lista de todas as ações possiveis nesse estado;
* `result(self, state, action)` - dados um estado e uma ação, este método devolve o estado resultante da execução da ação **action**, no estado **state**
* `goal_test(self, state)` - este método retornará `True` nos casos em que o estado fornecido (**state**) seja o estado final ou membro dos estados finais

Nesta aula iremos apenas trabalhar com a classe `ForwardPlan`.

### Espaços de estados: classe `ForwardPlan`

A classe `ForwardPlan` é uma sub-classe da classe `Problem`, isto significa que é necessário definir o goal e o estado inicial no construtor. No *ForwardPlan* o estado inicial corresponde ao conhecimento base do problema de planeamento e o goal aos objetivos. Vejamos brevemente o construtor desta classe:
```python
def __init__(self, planning_problem, verbose=False):
    super().__init__(associate('&', planning_problem.initial), associate('&', planning_problem.goals))
    self.planning_problem = planning_problem
    self.expanded_actions = self.planning_problem.expand_actions()
    if verbose:
        print(self.expanded_actions)
```
Reparem que a instrução `super().__init__` está a definir os atributos da classe *Problem* que esta sub-classe vai herdar, é aqui que definimos o estado inicial como sendo o conhecimento base (`planning_problem.initial`) e o goal como sendo o objetivo do problema de planeamento (`planning_problem.goals`). Para além disso, estamos a instanciar todas as ações viáveis (no atributo `expanded_actions`) a partir dos esquemas de ações definidos no problema de planeamento.

Recordemos o problema da torre dos três blocos definido anteriormente. Já definimos os estado inicial, objetivos, e ações de uma forma abstrata, recorrendo à classe `PlanningProblem`.

In [ ]:
# situação inicial
kb = 'On(A, Table) & On(B, Table) & On(C, A) & Clear(B) & Clear(C)'
# dominio
dominio = 'Block(A) & Block(B) & Block(C)'
# objetivo
goal = 'On(B, C) & On(A, B)'
# ações
acoes = [Action('MoveToTable(b, x)',
                precond='On(b, x) & Clear(b)',
                effect='On(b, Table) & Clear(x) & ~On(b, x)',
                domain='Block(b) & Block(x)'
               ),
         Action('Move(b, x, y)',
                precond='On(b, x) & Clear(b) & Clear(y)',
                effect='On(b, y) & Clear(x) & ~On(b, x) & ~Clear(y)',
                domain='Block(b) & Block(y)'
               )]

threeBlockTower = PlanningProblem(initial=kb, goals=goal, actions=acoes, domain=dominio)

Agora iremos transformar este problema de planeamento num problema de espaços de estados.

In [ ]:
p = ForwardPlan(threeBlockTower)

Ao instanciar um problema como espaços de estados, todas as ações viáveis serão geradas a partir dos esquemas de ações abstratos definidos no problema de planeamento.

Para verificarmos as ações geradas, temos de ativar o modo display.

In [ ]:
p = ForwardPlan(threeBlockTower, display=True)

Reparem que foram geradas todas as ações viáveis entre os pares de blocos e entre blocos e mesa. Além disso, nenhuma das ações  `MoveToTable(Table, x)`, `MoveToTable(x, Table)`, `Move(Table, x, y)` e `Move(x, y, Table)`, em que `x` e `y` são blocos A, B, C e Table, foi gerada. Este tipo de ações não foi gerado por causa do atributo dominio que existe no objeto `Action`. 

Existem quatro objetos definidos: A, B, C e Table, e todas as permutações destes objetos irão ser testadas em cada um dos esquemas de ações. No entanto, apenas algumas ações serão válidas por causa da filtragem do dominio (tal como explicado anteriormente), que só aceita alguns objetos como sendo blocos.

Vamos verificar o estado inicial e quais as ações possíveis para o estado inicial.

In [ ]:
p.initial

In [ ]:
a = p.actions(p.initial)
print(a)

Vamos aplicar uma ação e verificar o estado resultante

In [ ]:
e1 = p.result(p.initial,a[0])
print(e1)

##### Procura em espaços de estados

A procura `Forward` através dos espaços de estados, começa no estado inicial e usa as ações do problema para procurar "para a frente" um estado que satisfaça a expressão indicada no atributo `goal`.

Vamos aplicar a procura em largura para encontrar soluções para o nosso problema dos blocos `p`, problema definido anteriormente onde cada acção tem o domínio bem definido (são geradas 18 ações viáveis para serem usadas durante a procura, embora apenas um subconjunto delas é que pode satisfazer as pré-condições).

Usemos a função **breadth_first_graph_search_plus()**, que faz uma procura em largura em grafo e em que o objectivo é testado quando um estado é visitado, i.e., quando se geram os sucessores.

In [ ]:
blocks_world_solution = breadth_first_graph_search_plus(p).solution()
blocks_world_solution = list(map(lambda action: Expr(action.name, *action.args), blocks_world_solution))
print('Solução:',blocks_world_solution)

## Exercício 7

Temos duas cargas em dois aeroportos diferentes: carga 1 em Lisboa (LX) e carga 2 no Porto (OPO). O nosso objetivo é enviar cada carga para o outro aeroporto. Temos dois aviões para nos ajudar a completar essa tarefa. 

a) Formule este problema como um problema de planeamento.

**Dicas:**  
1. O dominio indica os vários objetos e os respetivos tipos, i.e., `Cargo(C1)` e `Plane(P1)` e `Airport(LX)` e ...
2. O objetivo será a carga C1 estar no Porto (OPO), `At(C1,OPO)` e a carga C2 estar em Lisboa (LX),  `At(C2,LX)`
3. O problema pode ser definido com três ações: 
    * `Load(c,p,a)` - carregar carga `c` no avião `p` que está no aeroport `a`, ie, `c` e `p`têm de estar ambos no aeroporto `a`. Esta ação terá o efeito de a carga estar dentro do avião (`In(c,p)`) e deixar de estar no aeroporto (`~At(c,a)`).
    * `Unload(c,p,a)` - descarregar carga `c` do avião `p` que está no aeroporto `a`. Esta ação terá o efeito de a carga estar no aeroporto (`At(c,a)`) e não estar dentro do avião (`~In(c,p)`).
    * `Fly(p,f,to)` - voo do avião `p` que parte de `f` (não se pode usar *from*, o planeador não aceita esse literal) com destino a `to`
4. Depois de definirem o problema podem aplicar o seguinte conjunto de ações que corresponde à solução do problema:
```python
solution = [expr("Load(C1 , P1, LX)"),
            expr("Fly(P1, LX, OPO)"),
            expr("Unload(C1, P1, OPO)"),
            expr("Load(C2, P2, OPO)"),
            expr("Fly(P2, OPO, LX)"),
            expr("Unload (C2, P2, LX)")]
```

b) Obtenha os tempos de execução e os paths (activando o modo display na função *breadth_first_graph_search_plus*, `display=True`) no Forward.

In [ ]:
# Resolução do exercicio 7



## Exercício 8

Pretendemos comprar alguns artigos: um pacote de leite, bananas, e um livro. Estamos inicialmente em casa e sabemos que o leite e as bananas estão disponíveis no supermercado, e o livro numa livraria. 

a) Formule este problema como um problema de planeamento.

b) Obtenha os tempos de execução e os paths (activando o modo display na função *breadth_first_graph_search_plus*, `display=True`) no Forward.

In [ ]:
# Resolução do exercicio 8



## Exercício 9

Consideremos uma torre de Hanoi com três discos (D1, D2 e D3), com buracos no centro e três pinos (A, B e C), onde os discos podem ser colocados. O disco D3 é maior que o disco D2, que é maior que o disco D1. Inicialmente, todos os discos estão no pino A, com D3 em baixo, D2 no meio e D1 no topo. Queremos movê-los para o pino C com a mesma configuração (D3 em baixo, D2 no meio e D1 no topo). As seguintes regras aplicam-se:
* Apenas o disco do topo num pino pode ser deslocado
* Um disco não pode ser colocado por cima de um disco menor  

a) Formule este problema.
<img src="hanoi.png" width="200">

b) Obtenha os tempos de execução e os paths (activando o modo display na função *breadth_first_graph_search_plus*, `display=True`) no Forward.

In [1]:
# Resolução do exercício 9

